# Recommendation system basic: matrix factorization

This notebook reviews/implements matrix factorization from scratch. 

Exmple application: movie recommendation 

Say we have data on user interaction metrics (say watching time ratio, the watching time/total movie length).

We have 3 movies and 4 users. 

In [124]:
import random
import numpy as np
SEED = 15
random.seed(SEED)
np.random.seed(SEED)

In [125]:
M1 = np.array([
        [0.1, 0.2, 0.9, 1.0],
        [0.1, 0.3, 0.8, 0.4],
        [1.0, 0.9, 0.1, 0.5],
    ])

In [140]:
def matrix_factorization(M, d, learning_rate=0.1, epochs=50, seed=15):
    """
    M = U * V^T
    M: original_matrix (size: m * n)
    U: item embedding (size: m * d)
    V: user embedding (size: n * d)
    
    Params:
        M(np.array)
        d(int): embedding dimension
        learning_rate(float): the rate to update weight in gradient descend
    Return:
        U(np.array)
        V(np.array)
    
    """
    np.random.seed(seed)
    
    m, n = M.shape
    # initialize U and V
    U = np.random.rand(m, d)
    V = np.random.rand(n, d)
    
    # gradient decent to update optimal U* and V*
    for epoch in range(epochs):
        # calculate current M
        M_pred = U @ V.T

        # caluclate error
        error = M - M_pred # m * n

        # update U and V
        U += learning_rate * error @ V
        V += learning_rate * error.T @ U

    return(U, V)
    

In [142]:
U, V = matrix_factorization(M1, 5, epochs=50)
print("Movie embedding")
print(U)
print("User embedding")
print(V)

Movie embedding
[[ 0.76575695  0.51821956 -0.15897315 -0.00906748  0.36561721]
 [ 0.61923485 -0.00444745  0.17655935 -0.09027221  0.1112234 ]
 [ 0.20730892 -0.06636708  0.36395578  1.05808794  0.45599005]]
User embedding
[[ 0.04363247 -0.00290447  0.34579585  0.64580841  0.396839  ]
 [ 0.26718721  0.07053684  0.77367716  0.42877172  0.24981469]
 [ 1.02412045  0.14344394  0.33230868 -0.34924897  0.31796322]
 [ 0.5525085   0.74612879  0.33089108  0.0521876   0.57116501]]


As a new user without historical data (commonly known as the cold start problem), there are two practical approaches:

1. Use the average of all users: This provides a general starting point but lacks personalization.
2. Incorporate user preferences: If the new user expresses a preference (e.g., liking Sci-Fi movies), we can identify existing users with similar interests and average their user embeddings to create a more tailored initialization.

In [146]:
# 1. Use the average of all users:
new_user_embedding1 = np.mean(V, axis=0)
new_user_embedding1

array([0.47186216, 0.23930128, 0.44566819, 0.19437969, 0.38394548])

In [147]:
# we can get the movie preference for the new user:
U @ new_user_embedding1.T

array([0.5531076 , 0.39497273, 0.62488915])

In [148]:
# 2. 
M1

array([[0.1, 0.2, 0.9, 1. ],
       [0.1, 0.3, 0.8, 0.4],
       [1. , 0.9, 0.1, 0.5]])

If we know that Movie 3 is a Sci-Fi movie, we can obtain this information in two ways:

1. Using metadata: The movie's genre can be directly retrieved from available metadata, such as tags, ㄎescriptions, or predefined categories.
2. Training a classification model: If genre labels are not explicitly available, we can train a model to classify movies based on their content features (e.g., text descriptions, embeddings, or user interactions).

From the matrix M1, we observe that User 1 and User 2 have high watching time ratios of 1.0 and 0.9, respectively. 
This suggests that these users have a strong preference for the movie.

To initialize a new user embedding, we can average the embeddings of these two similar users, assuming that the 
new user shares similar interests. This helps create a more personalized starting point for recommendations.

In [149]:
def get_new_user_embedding(M, V, preference_movie_idx, threshold=0.8):
    """
    Params:
        M(np.array): 
        V(np.array): user embedding from matrix factorization
        preference_movie_idx(list): the movie idx that matches the new user preference
        threshold(float): defines the minimum value required to consider a user as having a strong preference for a movie. 
                          If a user's value exceeds this threshold, their embedding can be used to approximate the new 
                          user's preferences.
    Returns:
        new_user_embedding1
    """
    # get the mask for preference_movie_idx
    row_mask = np.zeros(M.shape[0], dtype=bool)
    row_mask[preference_movie_idx] = True

    M_movie = M[row_mask]

    # 
    user_idx = np.where(M_movie > threshold)[1]

    # 
    new_user_embedding2 = V[user_idx, :].mean(axis=0)
    return(new_user_embedding2)
    
    

In [151]:
preference_movie_idx = [2]
new_user_embedding2 = get_new_user_embedding(M1, V, preference_movie_idx)
new_user_embedding2

array([0.15540984, 0.03381619, 0.5597365 , 0.53729006, 0.32332685])

In [152]:
# we can get the movie preference for the new user:
U @ new_user_embedding2.T

array([0.16088929, 0.18237065, 0.94962686])

Based on the results, we predict that the new user prefers Movie 3, with a preference score of up to 0.95.

As observed, Method 1 predicts values [0.550, 0.400, 0.625], which are centered around 0.5 and offer limited useful insights due to their similarity. Also, these predictions are shaped by existing user preferences, introducing bias, and fail to accurately reflect the new user's unique preferences. In contrast, Method 2 produces predictions that better reflect the new user's preferences, indicating that this approach may contribute to higher sales, improved customer retention, and enhanced brand reputation.

## incomplete input data

In the previous example, we had a complete movie-user interaction matrix (M1). However, in real-world scenarios, data is often sparse and incomplete, with many missing interactions.

In [153]:
M2 = np.array([
        [None, 0.2, None, 1.0],
        [0.1, None, None, 0.4],
        [1.0, 0.9, 0.1, None],
    ])

Several approaches can be used to handle incomplete data in matrix factorization:

1. Imputation: Fill in missing values using methods such as mean, median, max, or min imputers. However, this approach introduces significant bias, as the imputed values may not accurately reflect real user preferences.
2. Masking Missing Values: Instead of imputing missing values, apply matrix factorization while masking out missing entries. This ensures that only observed values contribute to the updates, improving model robustness and reducing bias.

In [154]:
def mask_matrix_factorization(M, d, learning_rate=0.1, epochs=50, seed=15):
    """
    M = U * V^T
    M: original_matrix (size: m * n)
    U: item embedding (size: m * d)
    V: user embedding (size: n * d)
    
    Params:
        M(np.array)
        d(int): embedding dimension
        learning_rate(float): the rate to update weight in gradient descend
    Return:
        U(np.array)
        V(np.array)
    
    """
    np.random.seed(seed)
    
    m, n = M.shape
    # initialize U and V
    U = np.random.rand(m, d)
    V = np.random.rand(n, d)

    # transform None to np.nan
    M = np.where(M == None, np.nan, M).astype(float)
    
    # mask
    mask = ~np.isnan(M)

    # assign 0 to np.nan
    M = np.nan_to_num(M)
    
    # gradient decent to update optimal U* and V*
    for epoch in range(epochs):
        # calculate current M
        M_pred = U @ V.T

        # caluclate error
        error = mask * (M - M_pred) # m * n

        # update U and V
        U += learning_rate * error @ V
        V += learning_rate * error.T @ U

    return(U, V)
    

In [156]:
U, V = mask_matrix_factorization(M2, 5, epochs=50)
print("Movie embedding")
print(U)
print("User embedding")
print(V)

Movie embedding
[[ 0.54887583  0.28140963 -0.21238567  0.44887535  0.43193131]
 [ 0.37827316  0.15772036  0.1479876  -0.06927787  0.00792639]
 [ 0.42127762  0.28437683  0.5431429   0.9509437   0.67049936]]
User embedding
[[ 0.25522108  0.19346783  0.18934407  0.44177014  0.46786456]
 [ 0.36172842  0.1800361   0.80561034  0.13101565  0.18758481]
 [ 0.63923519 -0.02147883  0.17331485 -0.37114707  0.14521031]
 [ 0.61644182  0.63327504  0.58949609  0.57617708  0.79223377]]


In [157]:
U @ V.T

array([[0.55469951, 0.21794123, 0.20412891, 0.99218145],
       [0.12818126, 0.27685824, 0.29092959, 0.38666511],
       [0.99917901, 0.89151172, 0.10174558, 1.83906671]])

In [158]:
M2

array([[None, 0.2, None, 1.0],
       [0.1, None, None, 0.4],
       [1.0, 0.9, 0.1, None]], dtype=object)

In [159]:
M1

array([[0.1, 0.2, 0.9, 1. ],
       [0.1, 0.3, 0.8, 0.4],
       [1. , 0.9, 0.1, 0.5]])

In [161]:
M1.sum(axis=1)

array([2.2, 1.6, 2.5])